# Librerias

In [3]:
import os
import sys
import comtypes.client
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# crear objeto API helper
helper = comtypes.client.CreateObject('ETABSv1.Helper')
helper = helper.QueryInterface(comtypes.gen.ETABSv1.cHelper)

myETABSObject = helper.CreateObjectProgID("CSI.ETABS.API.ETABSObject")     # Abrir una nueva ventana de Etabs
# myETABSObject = helper.GetObject("CSI.ETABS.API.ETABSObject")            # Trabajar con una ventana abierta de Etabs

myETABSObject.ApplicationStart()

# Crear un objeto SapModel
SapModel = myETABSObject.SapModel

# Iniciar Modelo
SapModel.InitializeNewModel()

# Crear un modelo en blanco
SapModel.File.NewBlank()

0

# Definicion de materiales

In [5]:
kgf_m_C = 8
SapModel.SetPresentUnits(kgf_m_C)

MATERIAL_CONCRETE = 2
SapModel.PropMaterial.SetMaterial('CONCRETO280', MATERIAL_CONCRETE)
E = 15000*(280**0.5)*10000   # (kgf/m2)
SapModel.PropMaterial.SetMPIsotropic('CONCRETO280', E, 0.2, 9.90E-06)
SapModel.PropMaterial.SetWeightAndMass('CONCRETO280', 1, 2400)
SapModel.PropMaterial.SetOConcrete('CONCRETO280', 2800000, False, 0, 1, 2, 0.0022, 0.0052)

MATERIAL_REBAR = 6
SapModel.PropMaterial.SetMaterial('REBAR', MATERIAL_REBAR)
SapModel.PropMaterial.SetMPIsotropic('REBAR', 2.0E10, 0.2, 9.90E-06)
SapModel.PropMaterial.SetWeightAndMass('REBAR', 1, 7850)
SapModel.PropMaterial.SetORebar('REBAR', 42000000, 63000000, 46000000, 69000000, 2, 2, 0.02, 0.1, False)

0

In [6]:
kgf_m_C = 8
SapModel.SetPresentUnits(kgf_m_C)

SapModel.PropFrame.SetRectangle('R1', 'CONCRETO280', 0.6, 0.3)     
SapModel.PropFrame.SetRebarBeam('R1','REBAR','REBAR', 0.06,0.06, 0, 0, 0, 0) 

0

In [7]:
FrameName1 = ' '
FrameName2 = ' '
FrameName3 = ' '
[FrameName1, _] = SapModel.FrameObj.AddByCoord(0, 0, 0, 0, 0, 10, FrameName1, 'R1')
[FrameName2, _] = SapModel.FrameObj.AddByCoord(0, 0, 10, 8, 0, 16, FrameName2, 'R1')
[FrameName3, _] = SapModel.FrameObj.AddByCoord(-4, 0, 10, 0, 0, 10, FrameName3, 'R1')

In [8]:
# kgf_m_C = 8
# SapModel.SetPresentUnits(kgf_m_C)
# SapModel.PropArea.SetSlab("Slab Ribbed", 3, 1, "CONCRETO280", 0.20)
# SapModel.PropArea.SetSlabRibbed("Slab Ribbed", 0.20, 0.005, 0.1, 0.1, 0.4, 1)               

# SapModel.PropArea.SetSlab("Losa Macisa", 0, 1, "CONCRETO280", 0.15)

# Asignar Restricciones

```python
Restraint = [True,      # U1
            True,       # U2
            True,       # U3
            True,       # R1
            False,      # R2
            False]      # R3

SapModel.PointObj.SetRestraint(PointName,       # Nombre del nodo a restringir
                               Restraint)       # Restricciones


```

```python
# Metodo clasico para obtener las coordenadas del nodo inicial y final de una elemento Frame

PointNamei = ' '
PointNamej = ' '

[PointNamei, PointNamej, ret] =  SapModel.FrameObj.GetPoints(FrameName,      # Nombre del elemento Frame
                                                             PointNamei,     # Nombre del nodo inicial:  " "
                                                             PointNamej)     # Nombre del nodo final:  " "


```

In [9]:
# Asignar resitricciones al punto de la Base
PointName1 = ' '
PointName2 = ' '
Restraint = [True, True, True, True, False, False]

[PointName1, PointName2, _] = SapModel.FrameObj.GetPoints(FrameName1, PointName1, PointName2)
SapModel.PointObj.SetRestraint(PointName1, Restraint)

# Asignar restricciones al punto superior
Restraint = [True, True, False, False, False, False]
[PointName1, PointName2, _] = SapModel.FrameObj.GetPoints(FrameName2, PointName1, PointName2)
SapModel.PointObj.SetRestraint(PointName2, Restraint)

[(True, True, False, False, False, False), 0]

In [12]:
Restraint = [True, True, True, True, True, True]
SapModel.PointObj.SetRestraint(' ', Restraint, 2)

[(True, True, True, True, True, True), 0]

# Crear LoadPattern

Definicion de Load Patterns

- Dead          = 1
- SuperDead     = 2
- Live          = 3
- ReduceLive    = 4
- Quake         = 5
- Wind          = 6
- Snow          = 7
- Other         = 8

In [13]:
SapModel.LoadPatterns.Add('1',      # Name del Load Pattern
                           1,       # Tipo de Load Pattern
                           1,       # Opcional - Considerar peso propio - Factor  default = 0
                           True)    # Opcional - True: Crear un nuevo caso de carga default = True

0

In [14]:
# Ejemplo - añadiendo LoadPatterns
LTYPE_OTHER = 8
SapModel.LoadPatterns.Add('2', LTYPE_OTHER)
SapModel.LoadPatterns.Add('3', LTYPE_OTHER)
SapModel.LoadPatterns.Add('4', LTYPE_OTHER)

0

In [15]:
## Ejemplos Extra
SapModel.LoadPatterns.Add("Carga Muerta",2)
SapModel.LoadPatterns.Add("Carga Viva Piso",4)  # Reducible Live
SapModel.LoadPatterns.Add("Carga Viva Techo",11) # Roof Live

SapModel.LoadPatterns.ChangeName("DEAD","Peso Propio")
SapModel.LoadCases.ChangeName("DEAD","Peso Propio")

0

## Mass Source

In [16]:
LoadPat = ["Peso Propio", 
           "Carga Muerta",
           "Carga Viva Piso", 
           "Carga Viva Techo"]
SF = [1.0, 
      1.0, 
      0.5, 
      0.25]
NumberLoads = len(LoadPat)
SapModel.PropMaterial.SetMassSource(3 , NumberLoads, LoadPat, SF)       # Para Etabs, por defecto se llamará "MsSrc1"

# SapModel.SourceMass.SetMassSource("MSSSRC1", False, False, True, True, 4, LoadPat, SF)              # Para Sap2000
# SapModel.SourceMass.ChangeName("MSSSRC1","Peso Sísmico")  # Cambiar el nombre del Mass Source       # Para Sap2000

[('Peso Propio', 'Carga Muerta', 'Carga Viva Piso', 'Carga Viva Techo'),
 (1.0, 1.0, 0.5, 0.25),
 0]

# Asignar Carga

Para carga distribuida:
1. Local 1 axis (only applies when CSys is Local)
2. Local 2 axis (only applies when CSys is Local)
3. Local 3 axis (only applies when CSys is Local)
4. X direction (does not apply when CSys is Local)
5. Y direction (does not apply when CSys is Local)
6. Z direction (does not apply when CSys is Local)
7. Projected X direction (does not apply when CSys is Local)
8. Projected Y direction (does not apply when CSys is Local)
9. Projected Z direction (does not apply when CSys is Local)
10. Gravity direction (only applies when CSys is Global)
11. Projected Gravity direction (only applies when CSys is Global)

```python
# carga puntual
                 
PointLoadValue = [0,    # Fx
                  0,    # Fy
                  -10,  # Fz
                  0,    # Mx
                  0,    # My
                  0]    # Mz                 
SapModel.PointObj.SetLoadForce(PointName1,          # Name of the point object
                                 '2',               # Load Pattern
                                 PointLoadValue)    # Values


# carga distribuida
SapModel.FrameObj.SetLoadDistributed(FrameName3,    # Nombre del Frame  
                                    '2',            # Load Pattern
                                    1,              # Tipo de carga distribuida  1 : F/L  2 : M/L
                                    2,              # Direccion de la carga   1-6 : Local      7-11 Global
                                    0,              # Distancia inicial de la carga  (relative por default)
                                    1,              # Distancia final de la carga    (relative por default)
                                    1.8,            # Valor inicial de la carga distribuida
                                    1.8,            # Valor final de la carga distribuida
                                    "Local")        # Sistema de coordenadas, por defecto viene Global
                                
    # existen más parametros a modificar pero estas son las principales
```

In [17]:
#assign loading for load pattern 2
PointName1 = ' '
PointName2 = ' '
[PointName1, PointName2, ret] = SapModel.FrameObj.GetPoints(FrameName3, PointName1, PointName2)

PointLoadValue = [0, 0,-10,0,0,0]
SapModel.PointObj.SetLoadForce(PointName1, '2', PointLoadValue)

[(0.0, 0.0, -10.0, 0.0, 0.0, 0.0), 0]

In [20]:
SapModel.FrameObj.SetLoadDistributed(FrameName2, '1', 1, 9, 0, 1, -0.3744, 0)

0

In [21]:
SapModel.FrameObj.SetLoadDistributed(FrameName3, '4', 1, 2, 0, 1, 1.8, 1.8, 'Local')
SapModel.FrameObj.SetLoadDistributed(FrameName2, '3', 1, 2, 0, 1, -0.3744, 0, 'Local')

0

```python
SapModel.FrameObj.SetLoadDistributed(FrameNamei,   # Nombre del Frame    
                                    '2',           # Load Pattern
                                    1,             # Tipo de carga distribuida  1 : F/L  2 : M/L
                                    10,            # Direccion de la carga   1-6 : Local      7-11 Global
                                    0,             # Distancia inicial de la carga  (relative por default)
                                    1,             # Distancia final de la carga    (relative por default)
                                    1.8,           # Valor inicial de la carga distribuida
                                    1.8,           # Valor final de la carga distribuida
                                    CSys,          # (Opcional) Default: Global       - Local   
                                    RelDist,       # (Opcional)   True: Si la distancia es relativa     False: Si la distancia es absoluta 
                                    Replace,       # (Opcional)   True: Para reemplazar las cargas que se asignan   
                                    ItemType)      # (Opcional)  2: Objeto seleccionado    

```


In [23]:
SapModel.FrameObj.SetLoadDistributed(" ", 'Carga Muerta', 1, 2, 0, 1, -10, -15, 'Local',True, True, 2)

0

# refinaciones

Asignar Brazo rigido       

```python
'assign offsets
SapModel.FrameObj.SetEndLengthOffset(FrameName,   # Nombre de un elemento Frame
                                    False,        # True: offset automatico  False: Para no asignar un offset al nodo inicial
                                    12, 12,       # Longitud 1 y 2
                                    0.5           # Factor de rigidez
                                    2)            # (Opcional)  2: Objeto seleccionado 
```

Asignar Releases 
```python

ii = [True,     # U1
      True,     # U2
      True,     # U3
      True,     # R1
      True,     # R2
      True]     # R3
jj = [True, True, True, True, True, True]    # Aplicacion de releases

StartValue = [0, # U1 partial fixity [F/L]
              0, # U2 partial fixity [F/L]
              0, # U3 partial fixity [F/L]
              0, # R1 partial fixity [M/L]
              0, # R2 partial fixity [M/L]
              0] # R3 partial fixity [M/L]    
EndValue = [0, 0, 0, 0, 0, 0]  # indican los resortes de fijación parcial del extremo del frame

ret =SapModel.FrameObj.SetReleases(FrameName,    # Nombre de un elemento Frame
                                    ii,          # Lista de booleanos para el nodo inicial
                                    jj,          # Lista de booleanos para el nodo final
                                    StartValue,  # Lista de valores de fijación parcial del nodo inicial
                                    EndValue,    # Lista de valores de fijación parcial del nodo final
                                    2)           # (Opcional)  2: Objeto seleccionado  
```
---

Asignar propiedad de una seccion a un frame seleccionado       
```python
'Crear una nueva propiedad de sección
 ret = SapModel.PropFrame.SetRectangle("R5", 
                                      "4000Psi", 
                                      20, 12)
'Asignar la propiedad de sección a un frame seleccionado
 ret = SapModel.FrameObj.SetSection(FrameName, 
                                    "R5",            # Nombre de la sección a asginar
                                    2)               # (Opcional)  2: Objeto seleccionado  
```
---